### Imports

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

#### Data imports

In [41]:
Base={
'Eolien en mer': 0.016740757290645353,
 'Eolien terrestre': 0.016345944160376588,
 'Fioul': 1.0413904849794613,
 'Charbon': 1.0970894476163344,
 'Autre renouvelables': 0.2346071813871875,
 'Centrales gaz': 0.626227083593243,
 'Nucleaire': 0.006463640014307128,
 'Hydraulique': 0.004332878603214016,
 'Solaire': 0.09029513144067704}
###############################################
SSP1_2030 = {
 'Eolien en mer': 0.012670743044279618,
 'Eolien terrestre': 0.011579548367147724,
 'Fioul': 1.0285618318056189,
 'Charbon': 1.1598831264430605,
 'Autre renouvelables': 0.21545680112137516,
 'Centrales gaz': 0.5885978510097314,
 'Nucleaire': 0.004988812230145272,
 'Hydraulique': 0.0032910774231261685,
 'Solaire': 0.04023441701061242}
#############################################
SSP1_2040 = {
 'Eolien terrestre': 0.009080462890491988,
 'Eolien en mer': 0.010289418289853882,
 'Fioul': 1.0233082792236114,
 'Charbon': 1.1390669973848309,
 'Autre renouvelables': 0.2076820592681813,
 'Centrales gaz': 0.5823343210808947,
 'Nucleaire': 0.003935716964806222,
 'Hydraulique': 0.0025361675299204168,
 'Solaire': 0.029829371268820147}
################################################
SSP1_2050 = {
 'Eolien terrestre': 0.008035543059599158,
 'Eolien en mer': 0.009219887304787615,
 'Fioul': 1.0200383591056437,
 'Charbon': 1.122012252277733,
 'Autre renouvelables': 0.20524946720835924,
 'Centrales gaz': 0.5818464786915487,
 'Nucleaire': 0.00335950883550386,
 'Hydraulique': 0.0019406765093110633,
 'Solaire': 0.02719571037823775}
#################################################
SSP2_2030 = {
 'Eolien terrestre': 0.010924169835351615,
 'Eolien en mer': 0.012147374016629606,
 'Fioul': 1.0274413801036677,
 'Charbon': 1.1567637408851734,
 'Autre renouvelables': 0.21183673271412792,
 'Centrales gaz': 0.5869865164520878,
 'Nucleaire': 0.004589613294724034,
 'Hydraulique': 0.0033636230223701956,
 'Solaire': 0.03543947684232527}
###################################################
SSP2_2040 = {
 'Eolien terrestre': 0.00894896900695643,
 'Eolien en mer': 0.010329255550794706,
 'Fioul': 1.0233814802269827,
 'Charbon': 1.1389795875657542,
 'Autre renouvelables': 0.20758283218945756,
 'Centrales gaz': 0.5814913985315437,
 'Nucleaire': 0.003968452749529839,
 'Hydraulique': 0.0025539592711940388,
 'Solaire': 0.029583252127025044}
####################################################
SSP2_2050 = {
 'Eolien terrestre': 0.0078066712878094085,
 'Eolien en mer': 0.009062180056060598,
 'Fioul': 1.0198914316533967,
 'Charbon': 1.121672330215476,
 'Autre renouvelables': 0.20498110669999614,
 'Centrales gaz': 0.5825042798300998,
 'Nucleaire': 0.0033356739942811474,
 'Hydraulique': 0.0018708649792158528,
 'Solaire': 0.018820714827474348}
#####################################################

# Conversion dicts
conversion_dicts = [Base,SSP1_2030,SSP1_2040,SSP1_2050,SSP2_2030,SSP2_2040,SSP2_2050]


In [42]:
dataset=xr.open_dataset('elec_mixes2020_2050.nc')
display(dataset)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 ...

### Characterize shares with CO2-eq

In [50]:
arrays_per_scenario = {}

for key1, dictionary in enumerate(conversion_dicts) : 
    list1 = []
    list2 = []

    for key2,val in dictionary.items() : 
        list1.append(key2)
        list2.append(val)

    # Make an xarray of the emission factors for each technology : 
    array = xr.DataArray(
        list2,
        coords = [list1],
        dims = ['technology']
    )

    # Multiply shares with individual technology emission amounts : 
    CO2_per_mix = dataset.copy()
    CO2_per_mix = CO2_per_mix*array

    # Store every cross arrays : 
    arrays_per_scenario[key1] = CO2_per_mix
# Sample display : 
display(arrays_per_scenario[0])

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

#### Slice the dataframes to correct years

In [52]:
scenario_ranges1 = {'Base':(0,2020,2025),'SSP1_2030':(1,2026,2035),'SSP1_2040':(2,2036,2045),'SSP1_2050':(3,2046,2050)}
scenario_ranges2 = {'Base':(0,2020,2025),'SSP2_2030':(4,2026,2035),'SSP2_2040':(5,2036,2045),'SSP2_2050':(6,2046,2050)}

In [ ]:
## For 13-10-2023 : try to "interpolate"

In [49]:
print(len(arrays_per_scenario))

7


In [63]:
scenario1_slices = {}
scenario2_slices = {}

for key,val in scenario_ranges1.items() : 
    sliced_array = arrays_per_scenario[val[0]].loc[dict(year=slice(val[1], val[2]))]
    scenario1_slices[key] = sliced_array
    print(key,val[0],val[1],val[2])

for key, val in scenario_ranges2.items() : 
    sliced_array = arrays_per_scenario[val[0]].loc[dict(year=slice(val[1],val[2]))]
    scenario2_slices[key] = sliced_array
    print(key,val[0],val[1],val[2])

display(scenario1_slices['SSP1_2030'])

Base 0 2020 2025
SSP1_2030 1 2026 2035
SSP1_2040 2 2036 2045
SSP1_2050 3 2046 2050
Base 0 2020 2025
SSP2_2030 4 2026 2035
SSP2_2040 5 2036 2045
SSP2_2050 6 2046 2050


<xarray.Dataset>
Dimensions:                        (Date: 365, year: 10, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.026e+03 ... 2.035e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

#### Patch everything back together for the two scenarios

In [64]:
scenario1 = xr.merge([scenario1_slices['Base'],scenario1_slices['SSP1_2030'],scenario1_slices['SSP1_2040'],scenario1_slices['SSP1_2050']])
scenario2 = xr.merge([scenario2_slices['Base'],scenario2_slices['SSP2_2030'],scenario2_slices['SSP2_2040'],scenario2_slices['SSP2_2050']])

In [62]:
display(scenario1)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

In [65]:
display(scenario2)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

### Export to netcdf

In [66]:
scenario1.to_netcdf(path = 'CO2_per_mix_S1.nc')
scenario2.to_netcdf(path = 'CO2_per_mix_S2.nc')

#### Validations

In [11]:
test_year = 2030
test_tech = 'Hydraulique'
test_date = 354
test_hour = 14

Valid_CO2=CO2_per_mix.sel(
    technology=test_tech,
    year= test_year,
    Date=test_date,
    Hour=test_hour,
).to_array().values[0]

Valid_share = dataset.sel(
    technology=test_tech,
    year= test_year,
    Date=test_date,
    Hour=test_hour,
).to_array().values[0]

print(test_tech,Valid_CO2,Valid_share*conversion_dicts[0][test_tech])

Hydraulique 0.0004423993060293206 0.0004423993060293206


In [22]:
for days in range(1,31) : 

    Valid_CO2=CO2_per_mix.sel(
        #technology=test_tech,
        year= 2025,
        Date=days,
        Hour=1,
    ).to_array().values[0].sum()
    print(Valid_CO2)

0.06298599564102884
0.05915633063768646
0.0458421118398691
0.05801437971609529
0.0685779249570528
0.05870453956431926
0.06692538550915995
0.05051387722669158
0.06420111981222341
0.053607577988887184
0.06052501890025235
0.04359380008043367
0.051857330795467074
0.04402511422306177
0.04408987503202299
0.061698094501918876
0.04757971212739592
0.07589700438810293
0.07299025261755367
0.07324306091117043
0.0840745782564879
0.0829861923146575
0.08349296150895148
0.08386240713188735
0.08224648274772453
0.07533303553523978
0.06234814862722442
0.06122405254078146
0.06679887781057725
0.065941104664966


In [4]:
# Extract the key-value pairs to list in an orderly manner : 
list1= []
list2 = []

for key, val in conversion_dicts[0].items() : 
        list1.append(key)
        list2.append(val)
    
# Make an xarray of the emission factors for each technology : 
array = xr.DataArray(
    list2,
    coords = [list1],
    dims = ['technology']
)

######################################
# Find a way to add a coordinate for Years (prevent the need for 3 netcdf final files (where each file would be only valid certain years)
# If so, just do all 3, then remove the invalid years per netcdf + merge all 3 after.
######################################


CO2_per_mix = dataset.copy()
CO2_per_mix = CO2_per_mix*array
display(CO2_per_mix)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 0.0...

In [5]:
CO2_per_mix.to_netcdf(path = 'CO2_per_mix.nc')